In [ ]:
%%bash
pip install matplotlib aiohttp

# 安装中文字体
yum install wqy-zenhei-fonts wqy-microhei-fonts fontconfig
# 检查字体是否安装成功
fc-list :lang=zh
fc-cache


In [ ]:
import matplotlib.pyplot as plt
from matplotlib import rcParams
from matplotlib import font_manager


# 设置字体为安装的中文字体（根据你的系统配置字体名称）
font_path = '/usr/share/fonts/wqy-zenhei/wqy-zenhei.ttc'  # 替换成你的字体路径
font_prop = font_manager.FontProperties(fname=font_path)
#rcParams['axes.unicode_minus'] = False  # 解决负号显示问题

# 测试中文显示
plt.plot([1, 2, 3], [1, 4, 9])
plt.title('中文标题')  # 中文标题
plt.xlabel('时间')  # 中文x轴标签
plt.ylabel('值')  # 中文y轴标签
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# 数据
x = np.linspace(0, 10, 100)
y1 = np.sin(x)
y2 = np.cos(x)

# 创建 2 行 1 列的子图
fig, axes = plt.subplots(2, 1, figsize=(8, 6))

# 第一个子图
axes[0].plot(x, y1, label='sin(x)')
axes[0].set_title('Sine Wave')
axes[0].set_xlabel('x')
axes[0].set_ylabel('y')
axes[0].legend()

# 第二个子图
axes[1].plot(x, y2, label='cos(x)', color='orange')
axes[1].set_title('Cosine Wave')
axes[1].set_xlabel('x')
axes[1].set_ylabel('y')
axes[1].legend()




In [ ]:
import asyncio
import aiohttp
import json
from functools import reduce

def get_stock_url(stock_code):
    base_url = 'https://29.push2.eastmoney.com/api/qt/stock/trends2/sse'
    query_params = {
        "fields1": "f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f17",
        "fields2": "f51,f52,f53,f54,f55,f56,f57,f58",
        "mpi": "1000",
        "ut": "fa5fd1943c7b386f172d6893dbfba10b",
        "secid": f"{stock_code}",
        "ndays": 1,
        "iscr": 0,
        "iscca": 0,
        "wbp2u": "|0|0|0|web"
        }
    query_string = reduce(lambda x,y:str(x)+'&'+str(y), map(lambda x:str(x)+'='+str(query_params.get(x, '')), query_params))
    url = f"{base_url}?{query_string}"
   
    return url

def get_url_headers(stock_code):
    stock_exchange = {
        "0": "sz",
        "1": "sh"
        }
    ex_key, code = stock_code.split(".")
    headers = {
        'Accept': 'text/event-stream',
        'Accept-Language':  'zh-CN,zh;q=0.9,en;q=0.8,en-GB;q=0.7,en-US;q=0.6',
        'Cache-Control':  'no-cache',
        'Connection':  'keep-alive',
        'Origin':  'https://quote.eastmoney.com',
        'Referer':  f'https://quote.eastmoney.com/{stock_exchange[ex_key]}{code}.html',
        'Sec-Fetch-Dest':  'empty',
        'Sec-Fetch-Mode':  'cors',
        'Sec-Fetch-Site':  'same-site',
        'User-Agent':  'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/133.0.0.0 Safari/537.36 Edg/133.0.0.0',
        'sec-ch-ua':  '"Not(A:Brand";v="99", "Microsoft Edge";v="133", "Chromium";v="133"',
        'sec-ch-ua-mobile':  '?0',
        'sec-ch-ua-platform':  '"Windows"'
        }
    return headers

def store_stock_data(stock_code, **kwargs):
    global stock_datas

def parse_data(stock_code, data):
    data = data[5:]
    data = json.loads(data)
    stock_data = data.get("data", {}) or {}
    
    stock_name = stock_data.get("name", "")
    trends_datas = stock_data.get("trends")
    if trends_datas:
        # print(trends_datas)
        lastest_trend_data = trends_datas[-1]
        period, open_price, close_price, high_price, \
            low_price, trad_volume, trad_amount, \
                average_price = trends_datas[-1].split(',')
        #print(f"当前{period}{stock_name}价格{open_price}~{close_price}")
        

def display_data(stock_codes):
    stock_counts = len(stock_codes)
    fig, axes = plt.subplots(stock_counts, 1)
    for i in range(stock_counts):
        axes[i].plot(x, y1, label='sin(x)')
        axes[0].set_title('Sine Wave')
        axes[0].set_xlabel('x')
        axes[0].set_ylabel('y')
        axes[0].legend()

    plt.tight_layout()
    plt.show()


async def get_stock_data(stock_code):
    url = get_stock_url(stock_code)
    headers = get_url_headers(stock_code)
    try:
        async with aiohttp.ClientSession() as session:
            async with session.get(url, headers=headers) as response:
                if response.status == 200:
                    async for line in response.content:
                        if line:
                            message = line.decode('utf-8').strip()
                            parse_data(stock_code, message)

                else:
                    print(f"Failed to connect to {url}, Status code: {response.status}")
    except Exception as e:
        print(f"Error while connecting to {url}: {e}")




"""
中公教育： 0.002607
洛阳钼业： 1.603993
"""
stock_codes = ["0.002607", "1.603993"]
tasks = [get_stock_data(code) for code in stock_codes]
# stock_data = {"code": {"open_price":[], "close_price":[], "average_price":[]}}
stock_datas = {}
await asyncio.gather(*tasks)

